In [2]:
from openai import OpenAI, AsyncOpenAI
import asyncio
import os
from datasets import load_dataset
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
from tqdm import tqdm   
import emoji
import json

In [3]:
def create_prompt(prompt,error,correct,wrong_token,correct_token):
    full_prompt = f'''{prompt}
Erroneous sentence: 
{error}
Correct sentence: 
{correct}'''
    return full_prompt


def save_results(temp, file_name):

    ## Initialize the file if it does not exist
    if not os.path.exists(file_name):
        with open(file_name,'w') as file:
            json.dump([],file)
    
    file_data = []
    with open(file_name,'r') as file:
        file_data = json.load(file)

    file_data.extend(temp)
    with open(file_name,'w') as file:
        # file.seek(0)
        json.dump(file_data,file)

In [4]:
import json

data_path = '../data/explanations_data/json/train.json'


data = []
with open(data_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

In [17]:
model_name = 'gpt-3.5-turbo'
# base_prompt='''
# The following is a sentence that contains only one grammatical error. The line after that contains the error word, and the line after that contains the correction.
# Please output a simple explanation of the error and the correction. After that, provide 3 examples of erroneous sentences and their corrections, that contain the same error and correction as the original sentence. Use this format for the examples:
# Error:
# Correction:
# '''
base_prompt='''The next two lines contain an erroneous sentence and its correction. Output a simple explanation of the error and the correction. After that, provide three examples that contain the same error.  Only output a valid JSON. The JSON must have these fields: explanation, example1_erroneous, example1_correct, example2_erroneous, example2_correct, example3_erroneous, and example3_correct.'''
chatgpt_outputs_file = f'../data/cahtgpt_outputs/{model_name}_outputs.json'
num_examples = 0
# num_examples = 2000

save_temps = []

offset = 0
# with open(chatgpt_outputs_file, 'a') as f:
for idx in tqdm(range(offset,num_examples)):
    
    # idx = idx + offset

    error = ' '.join(data[idx]['source']).replace('[NONE]', '').strip()
    correct = ' '.join(data[idx]['target']).replace('[NONE]', '').strip()

    concat = data[idx]['target'] + ['[SEP]'] + data[idx]['source']

    correct_token = concat[data[idx]['correction_index'][0]]#.replace('[NONE]', '').strip()
    wrong_token = concat[data[idx]['correction_index'][1]]#.replace('[NONE]', '').strip()


    final_prompt = create_prompt(base_prompt,error,correct,wrong_token,correct_token)
    
    try:
      # correct_answers.append(clue["target"])
      clue_message = {"role": "user", "content":final_prompt}
      completion = client.chat.completions.create(
        # request_timeout=15,
        model=model_name,
        response_format={"type" : "json_object"}, ## Add jsno output
        messages=[
          # system_message,
          clue_message
        ]
      )

      response = completion.choices[0].message.content.lower()
      response = json.loads(response)
      save_temps.append({'idx': idx, 'error': error,'correct': correct,'correct_token' : correct_token, 'wrong_token': wrong_token, 'chatgpt_response': response, 'prompt': final_prompt, 'model': model_name})
    except:
      save_temps.append({'idx': idx})
  

    if idx % 100 == 0 or idx == num_examples - 1:
      save_results(save_temps,chatgpt_outputs_file)
      save_temps = []


  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: 'source'

In [18]:
data[11690]

{'idx': 11690}

In [7]:
import json

with open('/home/abdelrahman.sadallah/mbzuai/UWFe/data/cahtgpt_outputs/gpt-3.5-turbo_outputs.json','r') as f:
    generated_data = json.load(f)

train_data = []
total = 0
cnt = 0
for l in generated_data:
    if 'chatgpt_response' not in l or    l['chatgpt_response'] == None:
        cnt += 1
        continue

    l['error_type'] = data[l['idx']]['error_type']


with open('/home/abdelrahman.sadallah/mbzuai/UWFe/data/cahtgpt_outputs/gpt3.5_generated_training.json','w') as f:
    json.dump(generated_data,f)


In [2]:
cnt, total, len(data)

(1, 15187, 15187)

In [6]:
data[15186].keys()

dict_keys(['target', 'source', 'evidence_index', 'correction_index', 'error_type', 'predicted_parsing_order', 'origin'])

In [8]:
import pandas as pd

df = pd.DataFrame(generated_data)

In [20]:
df['error_type'].value_counts()

error_type
Preposition                     2079
Verb Tense                      2046
Number                          1728
Collocation                     1609
Others                          1293
POS Confusion                   1084
Subject-Verb Agreement          1016
Article                          873
Possessive                       846
Gerund                           799
Infinitives                      723
Auxiliary Verb                   332
Pronoun-Antecedent Agreement     325
Transitive Verb                  259
Participle                       174
Name: count, dtype: int64

In [21]:
data[44]

{'target': ['But',
  ',',
  'the',
  'fact',
  'that',
  'it',
  'is',
  'short',
  'distance',
  'does',
  "n't",
  'mean',
  'it',
  'will',
  'be',
  'easy',
  'or',
  'anything',
  'like',
  'that',
  '.'],
 'source': ['But',
  ',',
  'the',
  'fact',
  'that',
  'it',
  'is',
  'short',
  'distance',
  'does',
  "n't",
  'mean',
  'it',
  'will',
  'be',
  'easy',
  'or',
  'something',
  'like',
  'that',
  '.'],
 'evidence_index': [],
 'correction_index': [17, 39],
 'error_type': 'Possessive',
 'predicted_parsing_order': {'11': 3,
  '15': 2,
  '17': 1,
  '18': 2,
  '19': 3,
  '20': 3,
  '33': 3,
  '37': 2,
  '39': 1,
  '40': 2,
  '41': 3,
  '42': 3},
 'origin': 'A'}